In [83]:
import numpy as np
from numpy import vstack
from scipy import integrate,stats

In [84]:
M = 10000
a_1 = 10.
b_1 = 1.


mu     = np.random.gamma(a_1, b_1, M)
beta   = np.random.gamma(a_1, b_1, M)
nu     = np.random.gamma(a_1, b_1, M)
sigma  = np.random.gamma(a_1, b_1, M)
kappa  = np.random.gamma(a_1, b_1, M)
K      = np.random.gamma(a_1, b_1, M)

mu_hat     = np.random.gamma(a_1, b_1, M)
beta_hat   = np.random.gamma(a_1, b_1, M)
nu_hat     = np.random.gamma(a_1, b_1, M)
sigma_hat  = np.random.gamma(a_1, b_1, M)
kappa_hat  = np.random.gamma(a_1, b_1, M)
K_hat      = np.random.gamma(a_1, b_1, M)

mu_m,mu_M       = 1e-5,0.1
beta_m,beta_M   = 1e-5,0.99
nu_m,nu_M       = 1e-5,0.1
sigma_m,sigma_M = 1e-4,0.9
kappa_m,kappa_M = 1e-6,1e-4
K_m,K_M         = 1e-2,4.

mu = (mu_M - mu_m)* ( mu - min(mu) ) / ( max(mu) - min(mu) ) + mu_m
beta = (beta_M - beta_m)* ( beta - min(beta) ) / ( max(beta) - min(beta) ) + beta_m
nu = (nu_M - nu_m)* ( nu - min(nu) ) / ( max(nu) - min(nu) ) + nu_m
sigma = (sigma_M - sigma_m)* ( sigma - min(sigma) ) / ( max(sigma) - min(sigma) ) + sigma_m
kappa = (kappa_M - kappa_m)* ( kappa - min(kappa) ) / ( max(kappa) - min(kappa) ) + kappa_m
K = (K_M - K_m)* ( K - min(K) ) / ( max(K) - min(K) ) + K_m
#print(beta)
mu_hat = (mu_M - mu_m)* ( mu_hat - min(mu_hat) ) / ( max(mu_hat) - min(mu_hat) ) + mu_m
beta_hat = (beta_M - beta_m)* ( beta_hat - min(beta_hat) ) / ( max(beta_hat) - min(beta_hat) ) + beta_m
nu_hat = (nu_M - nu_m)* ( nu_hat - min(nu_hat) ) / ( max(nu_hat) - min(nu_hat) ) + nu_m
sigma_hat = (sigma_M - sigma_m)* ( sigma_hat - min(sigma_hat) ) / ( max(sigma_hat) - min(sigma_hat) ) + sigma_m
kappa_hat = (kappa_M - kappa_m)* ( kappa_hat - min(kappa_hat) ) / ( max(kappa_hat) - min(kappa_hat) ) + kappa_m
K_hat = (K_M - K_m)* ( K_hat - min(K_hat) ) / ( max(K_hat) - min(K_hat) ) + K_m

In [85]:
A = np.column_stack((mu,beta, nu, sigma,kappa,K))
B = np.column_stack((mu_hat,beta_hat, nu_hat, sigma_hat,kappa_hat,K_hat))

In [86]:
numcols = len(A[0])
C_list = list()

i=0
while i < numcols:
    B_copy = np.copy(B)
    B_copy[:,i] = A[:,i]
    C_list.append(B_copy)
    i+=1

#print(len(C_list))

In [87]:
#This global index S_i is for total Incidence  respect to the first parameter (mu)
y_A=np.zeros((M, 1))
y_C=np.zeros((M, 1))
y_B=np.zeros((M, 1))


s=6 # 5 number of parameters
p=np.zeros((s, 1))
time = np.linspace(0.0,34.0,35)

# pesos para la cuadratura trapezoidal
weigths = np.ones(11)
weigths[0] = 0.5
weigths[-1] = 0.5


def modelo(x,t,p):
    """
    Entrada
    x - estado
    t -tiempo
    p - parámetros
    Salida
    fx - lado derecho de la ecuación diferencial parcial que define el modelo
    p[0]=mu,p[1]=beta,p[2]=nu,p[3]=sigma,p[4]=kappa, p[5]=K 
    """
  
    fx = np.zeros(3)
    xi=1e-5
    
    fx[0] = p[0] - ((p[1]*x[1]) + p[2]  + xi  )*x[0]
    fx[1] = ((p[1]*x[1])+ xi ) *x[0] - (p[3] + p[2])*x[1]
    fx[2] = p[3]*x[1] - (p[4] + p[2])*x[2]
    return fx


def solve(p):
    xi=1e-5
    x0 = np.array([1,0.00000001,0.00000194])
    nn = len(time)
    dt = 1.0/(10.0*nn)
    n_quad = 10.0*nn+1
    t_quad = np.linspace(time[0],time[-1],n_quad)
    soln = integrate.odeint(modelo,x0,t_quad,args=(p,))
    
    result = np.zeros(nn)
    
    for k in np.arange(35):   
        x_s = soln[10*k:10*(k+1)+1,0]
        x_i = soln[10*k:10*(k+1)+1,1]
        incidence = (p[1]*x_i+xi)*x_s       
        result[k] = dt*np.dot(weigths,incidence) 
        
    return p[5]*sum(result)

In [88]:
#This global index S_i is for total Incidence with respect to the first parameter (mu)
for i in range(M):    
    for k in range(s):
        p[k]=A[i][k]
        incidence_total_A=solve(p)
        y_A[i]=incidence_total_A
                
        
    for k in range(s):
        p[k]=B[i][k]
        incidence_total_B=solve(p)
        y_B[i]=incidence_total_B
        
    for k in range(s):
        p[k]=C_list[0][i][k] # 0 corresponds to mu
        incidence_total_C1=solve(p)
        y_C[i]=incidence_total_C1    
#    print(i)        
#    print(p)
           
#    print(y_C)

y_A_y_C=(1./M)*np.matmul(np.transpose(y_A), y_C)
y_A_y_A=(1./M)*np.matmul(np.transpose(y_A), y_A)
f_0_sq =( (1./M)*sum(y_A)  )*( (1./M)*sum(y_B)  )
var_E_Y_q_i= y_A_y_C - f_0_sq
var_Y      = y_A_y_A - f_0_sq
S_i        = var_E_Y_q_i/var_Y
print(y_A_y_C)
print(y_A_y_A)
print(f_0_sq)
print(var_E_Y_q_i)
print(var_Y)
print(S_i)

/home/kernel/.conda/envs/tf36/lib/python3.7/site-packages/ipykernel_launcher.py:43: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.


[[5.41799522e-05]]
[[0.00040589]]
[1.38272294e-05]
[[4.03527228e-05]]
[[0.00039206]]
[[0.10292486]]


In [89]:
#This global index S_i is for total Incidence with respect to the first parameter beta
for i in range(M):
        
    for k in range(s):
        p[k]=C_list[1][i][k] # 1 corresponds to beta
        incidence_total_C1=solve(p)
        y_C[i]=incidence_total_C1    

y_A_y_C=(1./M)*np.matmul(np.transpose(y_A), y_C)
y_A_y_A=(1./M)*np.matmul(np.transpose(y_A), y_A)
f_0_sq =( (1./M)*sum(y_A)  )*( (1./M)*sum(y_B)  )
var_E_Y_q_i= y_A_y_C - f_0_sq
var_Y      = y_A_y_A - f_0_sq
S_i        = var_E_Y_q_i/var_Y
print(y_A_y_C)
print(y_A_y_A)
print(f_0_sq)
print(var_E_Y_q_i)
print(var_Y)
print(S_i)

/home/kernel/.conda/envs/tf36/lib/python3.7/site-packages/ipykernel_launcher.py:43: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.


[[6.47330583e-05]]
[[0.00040589]]
[1.38272294e-05]
[[5.09058289e-05]]
[[0.00039206]]
[[0.12984193]]


In [90]:
#This global index S_i is for total Incidence with respect to the first parameter nu
for i in range(M):
        
    for k in range(s):
        p[k]=C_list[2][i][k] # 2 corresponds to nu
        incidence_total_C1=solve(p)
        y_C[i]=incidence_total_C1    

y_A_y_C=(1./M)*np.matmul(np.transpose(y_A), y_C)
y_A_y_A=(1./M)*np.matmul(np.transpose(y_A), y_A)
f_0_sq =( (1./M)*sum(y_A)  )*( (1./M)*sum(y_B)  )
var_E_Y_q_i= y_A_y_C - f_0_sq
var_Y      = y_A_y_A - f_0_sq
S_i        = var_E_Y_q_i/var_Y
print(y_A_y_C)
print(y_A_y_A)
print(f_0_sq)
print(var_E_Y_q_i)
print(var_Y)
print(S_i)

/home/kernel/.conda/envs/tf36/lib/python3.7/site-packages/ipykernel_launcher.py:43: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.


[[1.50884872e-05]]
[[0.00040589]]
[1.38272294e-05]
[[1.26125778e-06]]
[[0.00039206]]
[[0.003217]]


In [91]:
#This global index S_i is for total Incidence with respect to the first parameter sigma
for i in range(M):
        
    for k in range(s):
        p[k]=C_list[3][i][k] # 2 corresponds to sigma
        incidence_total_C1=solve(p)
        y_C[i]=incidence_total_C1    

y_A_y_C=(1./M)*np.matmul(np.transpose(y_A), y_C)
y_A_y_A=(1./M)*np.matmul(np.transpose(y_A), y_A)
f_0_sq =( (1./M)*sum(y_A)  )*( (1./M)*sum(y_B)  )
var_E_Y_q_i= y_A_y_C - f_0_sq
var_Y      = y_A_y_A - f_0_sq
S_i        = var_E_Y_q_i/var_Y
print(y_A_y_C)
print(y_A_y_A)
print(f_0_sq)
print(var_E_Y_q_i)
print(var_Y)
print(S_i)

/home/kernel/.conda/envs/tf36/lib/python3.7/site-packages/ipykernel_launcher.py:43: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.


[[2.59832646e-05]]
[[0.00040589]]
[1.38272294e-05]
[[1.21560352e-05]]
[[0.00039206]]
[[0.03100555]]


In [92]:
#This global index S_i is for total Incidence with respect to the first parameter kappa
for i in range(M):
        
    for k in range(s):
        p[k]=C_list[4][i][k] # 2 corresponds to kappa
        incidence_total_C1=solve(p)
        y_C[i]=incidence_total_C1    

y_A_y_C=(1./M)*np.matmul(np.transpose(y_A), y_C)
y_A_y_A=(1./M)*np.matmul(np.transpose(y_A), y_A)
f_0_sq =( (1./M)*sum(y_A)  )*( (1./M)*sum(y_B)  )
var_E_Y_q_i= y_A_y_C - f_0_sq
var_Y      = y_A_y_A - f_0_sq
S_i        = var_E_Y_q_i/var_Y
print(y_A_y_C)
print(y_A_y_A)
print(f_0_sq)
print(var_E_Y_q_i)
print(var_Y)
print(S_i)

/home/kernel/.conda/envs/tf36/lib/python3.7/site-packages/ipykernel_launcher.py:43: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.


[[1.21982016e-05]]
[[0.00040589]]
[1.38272294e-05]
[[-1.62902777e-06]]
[[0.00039206]]
[[-0.00415505]]


In [93]:
#This global index S_i is for total Incidence with respect to the first parameter K
for i in range(M):
        
    for k in range(s):
        p[k]=C_list[5][i][k] # 2 corresponds to K
        incidence_total_C1=solve(p)
        y_C[i]=incidence_total_C1    

y_A_y_C=(1./M)*np.matmul(np.transpose(y_A), y_C)
y_A_y_A=(1./M)*np.matmul(np.transpose(y_A), y_A)
f_0_sq =( (1./M)*sum(y_A)  )*( (1./M)*sum(y_B)  )
var_E_Y_q_i= y_A_y_C - f_0_sq
var_Y      = y_A_y_A - f_0_sq
S_i        = var_E_Y_q_i/var_Y
print(y_A_y_C)
print(y_A_y_A)
print(f_0_sq)
print(var_E_Y_q_i)
print(var_Y)
print(S_i)

/home/kernel/.conda/envs/tf36/lib/python3.7/site-packages/ipykernel_launcher.py:43: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.


[[1.82639374e-05]]
[[0.00040589]]
[1.38272294e-05]
[[4.43670803e-06]]
[[0.00039206]]
[[0.0113164]]
